# What affects methane chemical lifetime?

- methane
- VOCs
- NOx
- Ozone
- halocarbons (specifically ODSs)
- N2O
- climate

Here I suggest an override of the alpha scaling factor for methane that is calculated from multiple species.

Ozone itself is a function of other precursors: we do not include ozone as a direct influence on methane lifetime, and restrict ourselves to directly emitted anthropogenic species.

Gill Thornhill published two papers on methane lifetime: one on the chemical adjustments to lifetime, and one on the climate adjustments. Both effects will be included. We will 

1. take AerChemMIP multi-model means from Gill's papers
2. run the lifetime relationship to individual AerChemMIP models in Gill's papers
3. find a least squares fit with reasonable sensitivies across the historical USING UPDATES TO EMISSIONS AND CONCENTRATIONS
4. run a Monte Carlo that perturbs the sensitivity of lifetime to each emitted species

In [ ]:
from climateforcing.utils import mkdir_p
import numpy as np
import pandas as pd
import pooch
import matplotlib.pyplot as pl
import time
import scipy.stats
import scipy.optimize
from tqdm import tqdm

from fair import FAIR
from fair.interface import fill, initialise

In [ ]:
mkdir_p('../plots/')

## Temperature data

Use observations 1850-2020

In [ ]:
df_temp = pd.read_csv('../data/forcing/AR6_GMST.csv')
gmst = np.zeros(270)
gmst[0:101] = np.linspace(0, 0.1, 101)
gmst[100:270] = df_temp['gmst'].values[:170]+0.1

## Get emissions and concentrations 

In [ ]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)

rcmip_concentration_file = pooch.retrieve(
    url=(
        "doi:10.5281/zenodo.4589756/"
        "rcmip-concentrations-annual-means-v5-1-0.csv"
    ),
    known_hash="md5:0d82c3c3cdd4dd632b2bb9449a5c315f",
)

df_emis_rcmip = pd.read_csv(rcmip_emissions_file)
df_conc_rcmip = pd.read_csv(rcmip_concentration_file)
input = {}
hc_input = {}

In [ ]:
df_emis_update = pd.read_csv('../data/emissions/co2_ch4_n2o_slcfs_1750-2020.csv', index_col=0)
df_conc_update = pd.read_csv('../data/ar6_ghg_concentrations/wmghgs_1750-2019.csv', index_col=0)

In [ ]:
#df_conc_update

## Calibration

is from models, so use the CMIP6 emissions/concentrations for this step.

In [ ]:
conc_species = ['CH4', 'N2O']
hc_species = ['CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
               'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402']

for species in conc_species:
    input[species] = df_conc_rcmip.loc[
        (df_conc_rcmip['Scenario']=='ssp370') & (df_conc_rcmip['Variable'].str.endswith(species)) & 
        (df_conc_rcmip['Region']=='World'), '1750':'2019'
    ].interpolate(axis=1).values.squeeze()
    
for species in hc_species:
    species_rcmip_name = species.replace("-", "")
    hc_input[species] = df_conc_rcmip.loc[
        (df_conc_rcmip['Scenario']=='ssp370') & (df_conc_rcmip['Variable'].str.endswith(species_rcmip_name)) & 
        (df_conc_rcmip['Region']=='World'), '1750':'2019'
    ].interpolate(axis=1).values.squeeze()

In [ ]:
emis_species = ['CO', 'VOC', 'NOx']
for species in emis_species:
    input[species] = df_emis_rcmip.loc[
        (df_emis_rcmip['Scenario']=='ssp370') & (df_emis_rcmip['Variable'].str.endswith(species)) & 
        (df_emis_rcmip['Region']=='World'), '1750':'2020'
    ].interpolate(axis=1).values.squeeze()[:270]

In [ ]:
input['temp'] = gmst

In [ ]:
def calculate_eesc(
    concentration,
    baseline_concentration,
    fractional_release,
    fractional_release_cfc11,
    cl_atoms,
    br_atoms,
    br_cl_ratio = 45,
):

    # EESC is in terms of CFC11-eq
    eesc_out = (
        cl_atoms * (concentration - baseline_concentration) * fractional_release / fractional_release_cfc11 +
        br_cl_ratio * br_atoms * (concentration - baseline_concentration) * fractional_release / fractional_release_cfc11
    ) * fractional_release_cfc11
    return eesc_out

In [ ]:
fractional_release = {
    'CFC-11':0.47, 
    'CFC-12':0.23, 
    'CFC-113':0.29, 
    'CFC-114':0.12, 
    'CFC-115':0.04, 
    'HCFC-22':0.13, 
    'HCFC-141b':0.34, 
    'HCFC-142b':0.17,
    'CCl4':0.56, 
    'CHCl3':0, 
    'CH2Cl2':0, 
    'CH3Cl':0.44, 
    'CH3CCl3':0.67, 
    'CH3Br':0.6, 
    'Halon-1211':0.62,
    'Halon-1301':0.28, 
    'Halon-2402':0.65
}

cl_atoms = {
    'CFC-11':3, 
    'CFC-12':2, 
    'CFC-113':3, 
    'CFC-114':2, 
    'CFC-115':1, 
    'HCFC-22':1, 
    'HCFC-141b':2, 
    'HCFC-142b':1,
    'CCl4':4, 
    'CHCl3':3, 
    'CH2Cl2':2, 
    'CH3Cl':1, 
    'CH3CCl3':3, 
    'CH3Br':0, 
    'Halon-1211':1,
    'Halon-1301':0, 
    'Halon-2402':0
}

br_atoms = {
    'CFC-11':0, 
    'CFC-12':0, 
    'CFC-113':0, 
    'CFC-114':0, 
    'CFC-115':0, 
    'HCFC-22':0, 
    'HCFC-141b':0, 
    'HCFC-142b':0,
    'CCl4':0, 
    'CHCl3':0, 
    'CH2Cl2':0, 
    'CH3Cl':0, 
    'CH3CCl3':0, 
    'CH3Br':1, 
    'Halon-1211':1,
    'Halon-1301':1, 
    'Halon-2402':2
}

In [ ]:
hc_eesc = {}
total_eesc = 0
for species in hc_species:
    hc_eesc[species] = calculate_eesc(
        hc_input[species],
        hc_input[species][0],
        fractional_release[species],
        fractional_release['CFC-11'],
        cl_atoms[species],
        br_atoms[species],
    )
    total_eesc = total_eesc + hc_eesc[species]

In [ ]:
for species in hc_species:
    pl.plot(hc_eesc[species])

In [ ]:
input['HC'] = total_eesc

Use 1850 and 2014 emissions or concentrations corresponding to methane lifetime changes in Thornhill et al. 2021.

TODO: Could we also take into account the fact that there are multiple loss pathways for CH4:
- tropospheric OH loss is 560 Tg/yr
- chlorine oxidation, 11 Tg/yr, assumed not included in AerChemMIP models
- stratospheric loss is 31 Tg/yr, assumed not included in AerChemMIP models
- soil uptake, 30 Tg/yr, not included in AerChemMIP models

Saunois (2020): 90% of sink is OH chemistry in troposphere and is 553 [476–677] Tg CH4 yr−1, which is close to the IPCC number of 560, (chapter 5)

Chapter 6 only give time constants for soil uptake and the combined chemistry loss (trop OH + chlorine + stratosphere). 

In [ ]:
def alpha_scaling_exp(
    input,
    baseline,
    normalisation,
    beta,
):
    log_lifetime_scaling = 0
    for species in ['CH4', 'N2O', 'VOC', 'CO', 'HC', 'NOx', 'temp']:
        log_lifetime_scaling = log_lifetime_scaling + (
            np.log(1 + (input[species]-baseline[species])/normalisation[species] * beta[species])
        )
    return np.exp(log_lifetime_scaling)

In [ ]:
normalisation = {}
for species in ['CH4', 'N2O', 'VOC', 'CO', 'NOx', 'HC']:
    normalisation[species] = input[species][264] - input[species][100]
    print(species, normalisation[species])
normalisation['temp'] = 1

In [ ]:
baseline = {}
for species in ['CH4', 'N2O', 'VOC', 'CO', 'NOx', 'HC']:
    baseline[species] = input[species][100]
baseline['temp'] = 0

## Steps 1 and 2

Get and tune to AerChemMIP models

MRI and GISS both give pretty good historical emulations

In [ ]:
parameters = {}

parameters['AerChemMIP_mean'] = {
    'base': 10.0,
    'CH4': +0.22,
    'NOx': -0.33,
    'CO': 0,
    'VOC': +0.19,
    'HC': -0.037,
    'N2O': -0.02,
    'temp': -0.006,
}

parameters['UKESM'] = {
    'base': 8,
    'CH4': +0.22,
    'NOx': -0.25,
    'CO': 0,
    'VOC': +0.11,
    'HC': -0.049,
    'N2O': -0.012,
    'temp': -0.0408
}

# we'll exclude BCC and CESM as they don't have VOC expt and that's important. 
# We can live with a missing N2O from GFDL and a missing temperature feedback from MRI.

parameters['GFDL'] = {
    'base': 9.6,
    'CH4': +0.21,
    'NOx': -0.33,
    'CO': 0,
    'VOC': +0.15,
    'HC': -0.075,
    'N2O': 0,  # missing
    'temp': -0.0205
}

parameters['GISS'] = {
    'base': 13.4,
    'CH4': +0.18,
    'NOx': -0.46,
    'CO': 0,
    'VOC': +0.27,
    'HC': -0.006,
    'N2O': -0.039,
    'temp': -0.0333
}

parameters['MRI'] = {
    'base': 10.1,
    'CH4': +0.22,
    'NOx': -0.26,
    'CO': 0,
    'VOC': +0.21,
    'HC': -0.024,
    'N2O': -0.013,
    'temp': 0  # missing
}

In [ ]:
lifetime_scaling = {}

In [ ]:
models = ['AerChemMIP_mean', 'UKESM', 'GFDL', 'GISS', 'MRI']

In [ ]:
for model in models:
    print(parameters[model])
    lifetime_scaling[model] = alpha_scaling_exp(
        input,
        baseline,
        normalisation,
        parameters[model],
    )

In [ ]:
#pl.plot(np.arange(1750, 2501), aerchemmip_mean[:] * 8.25)
for model in models:
    pl.plot(np.arange(1750, 2020), lifetime_scaling[model] * parameters[model]['base'], label=model)
pl.legend(loc='upper left', bbox_to_anchor=[0, 0.8], frameon=False)
pl.xlim(1750, 2020)
pl.ylabel('CH4 chemical lifetime (yr)')
pl.title('CH$_4$ model calibration lifetime SSP3-7.0')
#pl.savefig('../plots/ch4_chemical_lifetime_tuning.png')

In [ ]:
# put this into a simple one box model
def one_box(
    emissions,
    gas_boxes_old,
    burden_per_emission,
    lifetime,
    alpha_lifetime,
    partition_fraction,
    pre_industrial_concentration,
    timestep=1,
    natural_emissions_adjustment=0,
):
    decay_rate = timestep/(lifetime * alpha_lifetime)
    decay_factor = np.exp(-decay_rate)
    
    # additions and removals
    gas_boxes_new = (
        partition_fraction
        * (emissions - natural_emissions_adjustment)
        * 1
        / decay_rate
        * (1 - decay_factor)
        * timestep
        + gas_boxes_old * decay_factor
    )

    airborne_emissions_new = gas_boxes_new
    concentration_out = (
        pre_industrial_concentration +
        burden_per_emission * airborne_emissions_new
    )
    return concentration_out, gas_boxes_new, airborne_emissions_new

For this model testing step, I'm now going to sub in emissions from observations

In [ ]:
# emis_ch4 = df_emis.loc[
#     (df_emis['Scenario']=='ssp370') & (df_emis['Variable'].str.endswith('CH4')) & 
#     (df_emis['Region']=='World'), '1750':'2500'
# ].interpolate(axis=1).values.squeeze()

emis_ch4_update = df_emis_update.loc['CH4'].values.squeeze()[:270]
conc_ch4_update = df_conc_update.loc['CH4'].values.squeeze()[:270]

In [ ]:
burden_per_emission = 1 / (5.1352e18 / 1e18 * 16.043 / 28.97)
partition_fraction = 1
pre_industrial_concentration = conc_ch4_update[0]
natural_emissions_adjustment = emis_ch4_update[0]

In [ ]:
conc_ch4 = {}

In [ ]:
for model in models:
    conc_ch4[model] = np.zeros(270)
    gas_boxes = 0
    airborne_emissions = 0
    for i in range(270):
        conc_ch4[model][i], gas_boxes, airborne_emissions = one_box(
            emis_ch4_update[i],
            gas_boxes,
            burden_per_emission,
            parameters[model]['base'],
            lifetime_scaling[model][i],
            partition_fraction,
            pre_industrial_concentration,
            timestep=1,
            natural_emissions_adjustment=natural_emissions_adjustment,
        )

In [ ]:
for model in models:
    pl.plot(np.arange(1750, 2020), conc_ch4[model], label=model)
pl.plot(np.arange(1750, 2020), conc_ch4_update, color='k', label='obs')
pl.ylabel('CH4 concentrations, ppb')
pl.xlim(1750, 2020)
pl.legend(frameon=False)
pl.title('CH$_4$ model calibrations SSP3-7.0')
#pl.savefig('../plots/aerchemmip_tuning_ch4_conc.pdf')
#pl.savefig('../plots/aerchemmip_tuning_ch4_conc.png')

In [ ]:
# df_ch4 = pd.read_csv('../data/calibration/methane_ukesm1.csv', index_col=0)

In [ ]:
# df_ch4.mean(axis=1)

In [ ]:
# pl.plot(np.arange(1850.5, 2015), conc_ch4['UKESM'][100:265], label='FaIR UKESM1 calibration')
# pl.plot(df_ch4.index, df_ch4.mean(axis=1), label='UKESM1')
# pl.legend()
# pl.savefig('../plots/ukesm_historical_methane.png')

In [ ]:
# for model in models:
#     pl.plot(np.arange(1750, 2021), conc_ch4[model][:271], label=model)
# pl.plot(np.arange(1750, 2021), input['CH4'][:271], color='k', label='obs')
# pl.legend()
# pl.savefig('../plots/aerchemmip_tuning_ch4_conc_1750-2020.pdf')

## Step 3

Find least squares sensible historical fit

Now we move to observational emissions and concentrations

In [ ]:
hc_eesc_update = {}
total_eesc_update = 0
for species in hc_species:
    hc_eesc_update[species] = calculate_eesc(
        df_conc_update.loc[species].values.squeeze()[:270],
        df_conc_update.loc[species].values.squeeze()[0],
        fractional_release[species],
        fractional_release['CFC-11'],
        cl_atoms[species],
        br_atoms[species],
    )
    total_eesc_update = total_eesc_update + hc_eesc_update[species]

In [ ]:
total_eesc_update

In [ ]:
input_update = {
    'CH4': conc_ch4_update,
    'NOx': df_emis_update.loc['NOx'].values.squeeze()[:270], 
    'VOC': df_emis_update.loc['VOC'].values.squeeze()[:270],
    'CO': df_emis_update.loc['CO'].values.squeeze()[:270],
    'HC': total_eesc_update,
    'N2O': df_conc_update.loc['N2O'].values.squeeze()[:270],
    'temp': input['temp']
}

In [ ]:
normalisation_update = {}
for species in ['CH4', 'N2O', 'VOC', 'CO', 'NOx', 'HC']:
    normalisation_update[species] = input_update[species][264] - input_update[species][100]
    print(species, normalisation_update[species])
normalisation_update['temp'] = 1

baseline_update = {}
for species in ['CH4', 'N2O', 'VOC', 'CO', 'NOx', 'HC']:
    baseline_update[species] = input_update[species][0]
baseline_update['temp'] = 0

In [ ]:
invect = np.array(
    [
        input_update['CH4'], 
        input_update['NOx'], 
        input_update['VOC'], 
        input_update['CO'], 
        input_update['HC'], 
        input_update['N2O'], 
        input['temp']
    ]
)

In [ ]:
def fit_precursors(x, rch4, rnox, rvoc, rco, rhc, rn2o, rtemp, rbase):
    conc_ch4 = np.zeros(270)
    gas_boxes = 0
    airborne_emissions = 0
    
    params = {}
    params['CH4'] = rch4
    params['NOx'] = rnox
    params['VOC'] = rvoc
    params['CO'] = rco
    params['HC'] = rhc
    params['N2O'] = rn2o
    params['temp'] = rtemp
    
    inp = {}
    inp['CH4'] = x[0]
    inp['NOx'] = x[1]
    inp['VOC'] = x[2]
    inp['CO'] = x[3]
    inp['HC'] = x[4]
    inp['N2O'] = x[5]
    inp['temp'] = x[6]
    
    lifetime_scaling = alpha_scaling_exp(
        inp,
        baseline_update,
        normalisation_update,
        params,
    )
    
    for i in range(270):
        conc_ch4[i], gas_boxes, airborne_emissions = one_box(
            emis_ch4_update[i],
            gas_boxes,
            burden_per_emission,
            rbase,
            lifetime_scaling[i],
            partition_fraction,
            pre_industrial_concentration,
            timestep=1,
            natural_emissions_adjustment=natural_emissions_adjustment,
        )
    return conc_ch4


p, cov = scipy.optimize.curve_fit(
    fit_precursors, 
    invect,
    conc_ch4_update[:270],
    bounds = (  # AerChemMIP min to max range
       (0.18, -0.46, 0.11, 0.11, -0.075, -0.039, -0.0408, 6.3),
       (0.26, -0.25, 0.27, 0.27, -0.006, -0.012, +0.0718, 13.4)
    )
)

In [ ]:
parameters['best_fit'] = {
    'base': p[7],
    'CH4': p[0],
    'NOx': p[1],
    'VOC': p[2],
    'CO': p[3],
    'HC': p[4],
    'N2O': p[5],
    'temp': p[6],
}
p

In [ ]:
# these are the feedback values per ppb / per Mt that go into FaIR
for specie in ['CH4', 'NOx', 'VOC', 'CO', 'HC', 'N2O']:
    print(specie, parameters['best_fit'][specie]/normalisation[specie])

In [ ]:
# beta_hc_sum = 0

# for species in hc_species:
#     beta_hc = (
#         p[3] * (
#             (hc_eesc[species][264] - hc_eesc[species][100])/(total_eesc[264]-total_eesc[100])
#         )
#     )
#     print(species, beta_hc)
#     beta_hc_sum = beta_hc_sum + beta_hc
# print(beta_hc_sum)

In [ ]:
input_update

In [ ]:
lifetime_scaling['best_fit'] = alpha_scaling_exp(
    input_update,
    baseline_update,
    normalisation_update,
    parameters['best_fit'],
)

In [ ]:
pl.plot(np.arange(1750, 2020), lifetime_scaling['best_fit'])

In [ ]:
lifetime_scaling['best_fit'][0]

In [ ]:
lifetime_scaling['best_fit'][0] * parameters['best_fit']['base']

In [ ]:
pl.plot(np.arange(1750, 2020), lifetime_scaling['best_fit'] * parameters['best_fit']['base'], label='best_fit')
pl.legend()
pl.ylabel('CH4 chemical lifetime (yr)')
#pl.savefig('../plots/ch4_chemical_lifetime_best_fit.pdf')

In [ ]:
conc_ch4['best_fit'] = np.zeros(270)
gas_boxes = 0
airborne_emissions = 0
for i in range(270):
    conc_ch4['best_fit'][i], gas_boxes, airborne_emissions = one_box(
        emis_ch4_update[i],
        gas_boxes,
        burden_per_emission,
        parameters['best_fit']['base'],
        lifetime_scaling['best_fit'][i],
        partition_fraction,
        conc_ch4_update[0],
        timestep=1,
        natural_emissions_adjustment=natural_emissions_adjustment,
    )

In [ ]:
pl.plot(np.arange(1750, 2020), conc_ch4['best_fit'][:], label='best_fit')
pl.plot(np.arange(1750, 2020), input_update['CH4'], color='k', label='observations')
pl.ylabel('CH4 concentrations, ppb')
pl.xlim(1750, 2020)
pl.title('CH$_4$ calibrated lifetime best coefficient fit')
pl.legend(frameon=False)
#pl.savefig('../plots/ch4_lifetime_best_fit_v_obs_ssp370.pdf')
#pl.savefig('../plots/ch4_lifetime_best_fit_v_obs_ssp370.png')

## Step 4

Run a Monte Carlo around these parameters

In [ ]:
samples = 100000

In [ ]:
param_mc = []
base = scipy.stats.uniform.rvs(6.3, 7.1, size=samples, random_state=1001251)
ch4 = scipy.stats.uniform.rvs(0.18, 0.08, size=samples, random_state=21260)
nox = scipy.stats.uniform.rvs(-0.46, 0.21, size=samples, random_state=3936801)
# voc and co are not separated in AR6, so rescale to preserve the mean of the sum
voc = scipy.stats.uniform.rvs(0.11-0.095, 0.16, size=samples, random_state=10372947)
co = scipy.stats.uniform.rvs(0.11-0.095, 0.16, size=samples, random_state=32015)
hc = scipy.stats.uniform.rvs(-0.075, 0.069, size=samples, random_state=9165539)
n2o = scipy.stats.uniform.rvs(-0.039, 0.027, size=samples, random_state=442935)
temp = scipy.stats.uniform.rvs(-0.0408, +0.1126, size=samples, random_state=1930159)  # should allow for wetland feedback
ch4_1850 = scipy.stats.norm.rvs(807.6, 13.8/1.65, size=samples, random_state=327801)
ch4_1750 = scipy.stats.norm.rvs(729.2, 9.4/1.65, size=samples, random_state=327801)
natural = scipy.stats.norm.rvs(153, 20, size=samples, random_state=2838698)
wetland = scipy.stats.norm.rvs(50, 10, size=samples, random_state=290193379)

for sample in range(samples):
    param_mc.append(
        {
            'base': base[sample],
            'CH4': ch4[sample],
            'NOx': nox[sample],
            'VOC': voc[sample],
            'CO': co[sample],
            'HC': hc[sample],
            'N2O': n2o[sample],
            'temp': temp[sample],
            'ch4_1750': ch4_1750[sample],
            'natural': natural[sample],
            'wetland': wetland[sample],
        }
    )

In [ ]:
731 * (np.exp(-1/lt))

In [ ]:
e

In [ ]:
e * 8 * (1 - np.exp(-1/8))

In [ ]:
lt = 8.8
e = 225
c = 731
vm = 0.351

def emis_to_conc(c, e, ts, lt, vm):
    return c * (np.exp(-ts/lt)) + ts * e * vm

for i in range(200):
    c = emis_to_conc(c, e, 1, lt, vm)
    print(c)

In [ ]:
def one_box_nat(
    emissions,
    concentration_in,
    burden_per_emission,
    lifetime,
    alpha_lifetime,
    timestep=1,
    natural_emissions=0,
):
    decay_rate = timestep/(lifetime * alpha_lifetime)
    decay_factor = np.exp(-decay_rate)
    
    # additions and removals
    concentration_out = (
        (emissions + natural_emissions)
        * 1
        / decay_rate
        * (1 - decay_factor)
        * burden_per_emission
        * timestep
        + concentration_in * decay_factor
    )

    return concentration_out

In [ ]:
conc_ch4_mc = {}
for sample in tqdm(range(samples)):
    conc_ch4_mc[sample] = np.zeros(270)
    gas_boxes = 0
    airborne_emissions = 0
    lifetime_scaling_mc = alpha_scaling_exp(
        input_update,
        baseline_update,
        normalisation_update,
        param_mc[sample],
    )
    conc_ch4_mc[sample][0] = param_mc[sample]['ch4_1750']
    for i in range(0,269):
        conc_ch4_mc[sample][i+1] = one_box_nat(
            emis_ch4_update[i],
            conc_ch4_mc[sample][i],
            burden_per_emission,
            param_mc[sample]['base'],
            lifetime_scaling_mc[i],
            timestep=1,
            natural_emissions=param_mc[sample]['natural'] + param_mc[sample]['wetland']*gmst[i],
        )

In [ ]:
def rmse(obs, mod):
    return np.sqrt(np.sum((mod-obs)**2))

In [ ]:
rms = np.zeros(100000)
for sample in range(100000):
    rms[sample] = rmse(input_update['CH4'], conc_ch4_mc[sample])

In [ ]:
np.sum(rms < 600)

In [ ]:
np.argmin(rms)

In [ ]:
pl.plot(np.arange(1750, 2020), conc_ch4_mc[np.argmin(rms)])
pl.plot(np.arange(1750, 2020), input_update['CH4'], color='k')

In [ ]:
accept = rms<600
#accept = np.zeros(samples, dtype=bool)
#for sample in range(samples):
    #if input_update['CH4'][169] - 10 < conc_ch4_mc[sample][169] < input_update['CH4'][169] + 10:
    #    accept[sample] = True

In [ ]:
for sample in range(samples):
    if accept[sample]:
        pl.plot(np.arange(1750, 2020), conc_ch4_mc[sample])
pl.plot(np.arange(1750, 2020), input_update['CH4'], color='k')

In [ ]:
param_mc[5554]

In [ ]:
accept

In [ ]:
param_mc[9999]

In [ ]:
accept_inds = np.arange(samples, dtype=int)[accept]
accept_inds

In [ ]:
base_accept = np.zeros_like(accept_inds, dtype=float)
ch4_accept = np.zeros_like(accept_inds, dtype=float)
nox_accept = np.zeros_like(accept_inds, dtype=float)
co_accept = np.zeros_like(accept_inds, dtype=float)
voc_accept = np.zeros_like(accept_inds, dtype=float)
for i, n in enumerate(accept_inds):
    base_accept[i] = param_mc[n]['base']
    ch4_accept[i] = param_mc[n]['CH4']
    nox_accept[i] = param_mc[n]['NOx']
    co_accept[i] = param_mc[n]['CO']
    voc_accept[i] = param_mc[n]['VOC']

In [ ]:
pl.hist(co_accept)

In [ ]:
pl.hist(voc_accept)

In [ ]:
pl.hist(co_accept+voc_accept)

In [ ]:
pl.plot(emis_ch4_update-emis_ch4_update[0])
pl.plot(conc_ch4_update-conc_ch4_update[0])

In [ ]:
pl.plot((emis_ch4_update[50:]-emis_ch4_update[0])/(conc_ch4_update[50:]-conc_ch4_update[0]))

In [ ]:
emis_ch4_update[0]

In [ ]:
conc_ch4_update[0]   # ppb = volume * 1e-9

In [ ]:
burden_per_emission  # ppb per MtCH4 emitted

In [ ]:
emis_ch4_update[0] * burden_per_emission  #  ppb increase for one year of 1750 emissions

In [ ]:
# lifetime - with no natural emissions (obviously incorrect)
conc_ch4_update[0] / (emis_ch4_update[0] * burden_per_emission)

In [ ]:
# lifetime - with 200 MtCH4 natural emissions
conc_ch4_update[0] / ((emis_ch4_update[0]+200) * burden_per_emission)

In [ ]:
# what would the lifetime be if we assumed a constant 200 MtCH4 background natural emissions flux?
conc_ch4_update / ((emis_ch4_update+200) * burden_per_emission)

In [ ]:
# add in a wetland feedback!
pl.plot(np.arange(1750, 2020), conc_ch4_update / ((emis_ch4_update+160+40*gmst) * burden_per_emission))

In [ ]:
pl.plot(input_update['N2O'])